In [ ]:
### Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

### Load Dataset
data = pd.read_csv('/kaggle/input/modeldata/data1.csv')

### Preprocessing
# Separate features and labels
X = data.drop('label', axis=1)
y = data['label']

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

### Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Reshape for GRU Input
# GRU requires 3D input: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

### Build and Evaluate Multiple GRU Models with Different Hyperparameters
def build_and_evaluate_model(learning_rate, layers):
    model = Sequential()

    # Add GRU layers
    for i, units in enumerate(layers):
        if i == 0:
            model.add(GRU(units, activation='tanh', return_sequences=(i != len(layers) - 1), input_shape=(1, X_train.shape[2])))
        else:
            model.add(GRU(units, activation='tanh', return_sequences=(i != len(layers) - 1)))
        model.add(Dropout(0.3))

    # Add Dense layers
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(np.unique(y)), activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Early Stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy with learning_rate={learning_rate} and layers={layers}: {accuracy * 100:.2f}%")
    return accuracy

### Experiment with Different Configurations
learning_rates = [0.001, 0.0005, 0.0001]
layer_configurations = [[128, 64], [256, 128, 64], [128, 128, 64, 32]]

best_accuracy = 0
best_config = None

for lr in learning_rates:
    for layers in layer_configurations:
        print(f"Evaluating model with learning_rate={lr} and layers={layers}...")
        accuracy = build_and_evaluate_model(lr, layers)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_config = (lr, layers)

print(f"Best Accuracy: {best_accuracy * 100:.2f}% with learning_rate={best_config[0]} and layers={best_config[1]}")


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the CSV file
file_path = 'corrected_data1.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(data.head())

# Assuming 'label' is the target column and the rest are features
X = data.drop(columns=['label'])
y = data['label']

# Encode labels to integers if they are categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define layer configurations and learning rates
layer_configurations = [[128, 64], [256, 128, 64]]
learning_rates = [0.01, 0.001]

# Function to create and compile a model
def create_model(layer_config, learning_rate):
    model = Sequential()
    model.add(Dense(layer_config[0], activation='relu', input_shape=(X_train_scaled.shape[1],)))
    for units in layer_config[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models with different configurations
for layer_config in layer_configurations:
    for learning_rate in learning_rates:
        print(f"Training model with layers {layer_config} and learning rate {learning_rate}")
        model = create_model(layer_config, learning_rate)
        history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)
        test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
        print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
        print("-" * 50)

       Frequency         Z     Deg       Z.1   Deg.1       Z.2   Deg.2  \
0      10.000000   85042.0 -76.231   89189.0 -77.743   90240.0 -77.253   
1  378314.728446  138389.0 -14.992  132932.0 -15.101  130529.0 -15.183   
2  366799.267633  138709.0 -15.037  133246.0 -15.163  130821.0 -15.254   
3  355634.324069  139043.0 -15.088  133551.0 -15.234  131141.0 -15.332   
4  344809.228416  139379.0 -15.154  133867.0 -15.314  131457.0 -15.421   

          Z_mean  Z_median  Z_weighted_avg  Z_geometric_mean  Z_harmonic_mean  \
0   88157.000000   89189.0    88214.122097      88128.142301     88098.997497   
1  133950.000000  132932.0   134030.737305     133909.906545    133870.100205   
2  134258.666667  133246.0   134339.725401     134218.411726    134178.443200   
3  134578.333333  133551.0   134659.584458     134537.985845    134497.928443   
4  134901.000000  133867.0   134982.498719     134860.530591    134820.353189   

    Deg_mean  Deg_median  Deg_weighted_avg    label  
0 -77.075667  

c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0067 - loss: -615092992.0000 - val_accuracy: 0.0076 - val_loss: -9377690624.0000
Epoch 2/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0065 - loss: -18390005760.0000 - val_accuracy: 0.0076 - val_loss: -59433259008.0000
Epoch 3/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0061 - loss: -80922615808.0000 - val_accuracy: 0.0076 - val_loss: -164024090624.0000
Epoch 4/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.0058 - loss: -201651421184.0000 - val_accuracy: 0.0076 - val_loss: -331696078848.0000
Epoch 5/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0064 - loss: -385831010304.0000 - val_accuracy: 0.0076 - val_loss: -572631810048.0000
Epoch 6/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0063 - loss: -648411938816.0000 - val_accuracy: 0.0076 - val_loss: -896983105536.0000
Epoch 7/50
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0067 - loss: -98